# Running a decision tree

In [30]:
# -*- coding: utf-8 -*-

from pandas import Series, DataFrame
from pandas import *
import numpy as np
import os
import matplotlib.pylab as plt
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import sklearn.metrics

#Load the dataset
data = read_csv("data/marscrater_pds.csv")
data.dropna(inplace=True)
data.head()

,﻿CRATER_ID,CRATER_NAME,LATITUDE_CIRCLE_IMAGE,LONGITUDE_CIRCLE_IMAGE,DIAM_CIRCLE_IMAGE,DEPTH_RIMFLOOR_TOPOG,MORPHOLOGY_EJECTA_1,MORPHOLOGY_EJECTA_2,MORPHOLOGY_EJECTA_3,NUMBER_LAYERS
0,01-000000,,84.367,108.746,82.10,0.22,,,,0
1,01-000001,Korolev,72.760,164.464,82.02,1.97,Rd/MLERS,HuBL,,3
2,01-000002,,69.244,-27.240,79.63,0.09,,,,0
3,01-000003,,70.107,160.575,74.81,0.13,,,,0
4,01-000004,,77.996,95.617,73.53,0.11,,,,0


In [31]:
data.columns

Index([u'﻿CRATER_ID', u'CRATER_NAME', u'LATITUDE_CIRCLE_IMAGE',
       u'LONGITUDE_CIRCLE_IMAGE', u'DIAM_CIRCLE_IMAGE',
       u'DEPTH_RIMFLOOR_TOPOG', u'MORPHOLOGY_EJECTA_1', u'MORPHOLOGY_EJECTA_2',
       u'MORPHOLOGY_EJECTA_3', u'NUMBER_LAYERS'],
      dtype='object')

In [32]:
data.dtypes

﻿CRATER_ID                 object
CRATER_NAME                object
LATITUDE_CIRCLE_IMAGE     float64
LONGITUDE_CIRCLE_IMAGE    float64
DIAM_CIRCLE_IMAGE         float64
DEPTH_RIMFLOOR_TOPOG      float64
MORPHOLOGY_EJECTA_1        object
MORPHOLOGY_EJECTA_2        object
MORPHOLOGY_EJECTA_3        object
NUMBER_LAYERS               int64
dtype: object

In [33]:
data.describe()

,LATITUDE_CIRCLE_IMAGE,LONGITUDE_CIRCLE_IMAGE,DIAM_CIRCLE_IMAGE,DEPTH_RIMFLOOR_TOPOG,NUMBER_LAYERS
count,384343.000000,384343.000000,384343.000000,384343.000000,384343.000000
mean,-7.199209,10.128021,3.556686,0.075838,0.064835
std,33.608966,96.641592,8.591993,0.221518,0.304889
min,-86.700000,-179.997000,1.000000,-0.420000,0.000000
25%,-30.935000,-58.829000,1.180000,0.000000,0.000000
50%,-10.079000,12.740000,1.530000,0.000000,0.000000
75%,17.222500,89.275500,2.550000,0.000000,0.000000
max,85.702000,179.997000,1164.220000,4.950000,5.000000


In [34]:
## our target is going to be whether or not the crater is large
## we're defining large diameter craters as greater than 2 km
data['LARGE_DIAM'] = data.DIAM_CIRCLE_IMAGE >= 2.0

In [40]:
#Split into training and testing sets

predictors = data[['LATITUDE_CIRCLE_IMAGE', 'LONGITUDE_CIRCLE_IMAGE', 'DEPTH_RIMFLOOR_TOPOG', 'NUMBER_LAYERS']]

targets = data.LARGE_DIAM

X_train, X_test, y_train, y_test  =   train_test_split(predictors, targets, test_size=.4)

X_train.shape

(230605, 4)

In [41]:
X_test.shape

(153738, 4)

In [42]:
y_train.shape

(230605L,)

In [43]:
y_test.shape

(153738L,)

In [51]:
#Build model on training data
classifier=DecisionTreeClassifier()
classifier=classifier.fit(X_train,y_train)

predictions=classifier.predict(X_test)

confmatrix = sklearn.metrics.confusion_matrix(y_test,predictions)
confmatrix = DataFrame(data=confmatrix, 
                       columns=['Predicted small crater', 'Predicted large crater'],
                       index=['Actual small crater', 'Actual large crater'])
confmatrix

,Predicted small crater,Predicted large crater
Actual small crater,82205,19047
Actual large crater,18192,34294


In [53]:
sklearn.metrics.accuracy_score(y_test, predictions)

0.75777621668032624

In [ ]:
# -*- coding: utf-8 -*-
#Displaying the decision tree
from sklearn import tree
from io import BytesIO
from IPython.display import Image
out = BytesIO()
tree.export_graphviz(classifier, out_file=out)
import pydotplus
graph=pydotplus.graph_from_dot_data(out.getvalue())
Image(graph.create_png())